# Reindeer Maze

In [11]:
import json
import strings

EMPTY = "."
FENCE = "#"
START = "S"
FINISH = "E"

DIRECTIONS = {
    "^": (0, -1),
    ">": (1, 0),
    "v": (0, 1),
    "<": (-1, 0),
}

def get_matrix(area: str):
    return list(map(lambda line: list(line), area.split("\n")))

def render_matrix(matrix):
    return "\n".join(["".join(line) for line in matrix])

def get_position(matrix, element):
    for y in range(len(matrix)):
        try:
            x = matrix[y].index(element)
            return (x, y)
        except ValueError:
            continue

def get_xy(position, direction):
    x, y = position
    dx, dy = DIRECTIONS[direction]
    return (x + dx, y + dy)

def get_value(matrix, position):
    x, y = position
    return matrix[y][x]

def is_gateway(matrix, position):
    if get_value(matrix, position) not in [EMPTY, START, FINISH]:
        return False

    # corners
    if (get_value(matrix, get_xy(position, "^")) == FENCE 
        and get_value(matrix, get_xy(position, "<")) == FENCE
        and get_value(matrix, get_xy(position, "v")) == EMPTY
        and get_value(matrix, get_xy(position, ">")) == EMPTY
        ):
        return True

    if (get_value(matrix, get_xy(position, "^")) == FENCE 
        and get_value(matrix, get_xy(position, ">")) == FENCE
        and get_value(matrix, get_xy(position, "v")) == EMPTY
        and get_value(matrix, get_xy(position, "<")) == EMPTY
        ):
        return True

    if (get_value(matrix, get_xy(position, "v")) == FENCE 
        and get_value(matrix, get_xy(position, ">")) == FENCE
        and get_value(matrix, get_xy(position, "^")) == EMPTY
        and get_value(matrix, get_xy(position, "<")) == EMPTY
        ):
        return True

    if (get_value(matrix, get_xy(position, "v")) == FENCE 
        and get_value(matrix, get_xy(position, "<")) == FENCE
        and get_value(matrix, get_xy(position, "^")) == EMPTY
        and get_value(matrix, get_xy(position, ">")) == EMPTY
        ):
        return True

    # cross road
    if (get_value(matrix, get_xy(position, "v")) == EMPTY 
        and get_value(matrix, get_xy(position, "<")) == EMPTY
        and get_value(matrix, get_xy(position, "^")) == EMPTY
        and get_value(matrix, get_xy(position, ">")) == EMPTY
        ):
        return True

    # single side
    if (get_value(matrix, get_xy(position, "v")) == FENCE 
        and get_value(matrix, get_xy(position, "<")) == EMPTY
        and get_value(matrix, get_xy(position, "^")) == EMPTY
        and get_value(matrix, get_xy(position, ">")) == EMPTY
        ):
        return True

    if (get_value(matrix, get_xy(position, "v")) == EMPTY
        and get_value(matrix, get_xy(position, "<")) == FENCE
        and get_value(matrix, get_xy(position, "^")) == EMPTY
        and get_value(matrix, get_xy(position, ">")) == EMPTY
        ):
        return True

    if (get_value(matrix, get_xy(position, "v")) == EMPTY 
        and get_value(matrix, get_xy(position, "<")) == EMPTY
        and get_value(matrix, get_xy(position, "^")) == FENCE
        and get_value(matrix, get_xy(position, ">")) == EMPTY
        ):
        return True

    if (get_value(matrix, get_xy(position, "v")) == EMPTY 
        and get_value(matrix, get_xy(position, "<")) == EMPTY
        and get_value(matrix, get_xy(position, "^")) == EMPTY
        and get_value(matrix, get_xy(position, ">")) == FENCE
        ):
        return True

    return False
        


def find_gateways(matrix):
    out = []
    for y in range(1, len(matrix) -1):
        for x in range(1, len(matrix[y])-1):
            if is_gateway(matrix, (x, y)):
                out.append((x,y))
    return out

class Node():
    parent = None
    childs = None # []
    weight = 0
    
    def __init__(self, position, parent=None, weight=0):
        self.parent = parent
        self.position = position
        self.weight = weight
        self.childs = []

    def get_next_point(self, matrix, gateways, position, vector, distance=0):
        x, y = position
        dx, dy = vector
        
        if get_value(matrix, position) == FENCE:
            return None
        next_position = (x+dx, y+dy)
        if next_position in gateways:
            return next_position, distance+1

        return self.get_next_point(matrix, gateways, next_position, vector, distance+1)
        
    def make_child(self, position, weight):
        child = Node(position, parent=self, weight=weight)
        self.childs.append(child)
        return child
    
    def make_childs(self, matrix, gateways):
        gw = gateways
        x, y = self.position

        for vector in [(0, -1),(1, 0),(0, 1),(-1, 0)]:
            child = self.get_next_point(matrix, gw, self.position, vector)
            if child == None:
                continue

            position, distance = child
            gw.remove(position)
            node = self.make_child(position, weight=distance)
            node.make_childs(matrix, gw)

    def serialise(self):
        return {
            "p": self.position,
            "w": self.weight,
            "b": [
                child.serialise() for child in self.childs
            ]
        }

    def serialize_flat(self):
        return [(self.position, self.weight), *[item for child in self.childs for item in child.serialize_flat()]]
        

def make_graph(matrix, gateways):
    sx, sy = get_position(matrix, START)
    ex, ey = get_position(matrix, FINISH)
    start_node = Node((sx,sy))
    start_node.make_childs(matrix, [*gateways])

    return start_node.serialize_flat()

# Manhattan distance
def h(current, finish):
    cx, cy = current
    fx, fy = finish
    return abs(cx - fx) + abs(cy - fx)

with open("test.txt", "r") as f:
    area = f.read().strip()

matrix = get_matrix(area)

print(render_matrix(matrix))
print()

gateways = find_gateways(matrix)
clone = get_matrix(area)
for x,y in gateways:
    clone[y][x] = "O"
print(render_matrix(clone))


clone = get_matrix(area)
for position, weight in make_graph(matrix, gateways):
    x, y = position
    clone[y][x] = "x"
    print(render_matrix(clone))
    print()





###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############

###############
#O.O...O#O...O#
#.#.###.#.###.#
#O.O.O#.#O.O#.#
#.###.#####.#.#
#.#.#O...O.O#.#
#.#.#####.###.#
#O.O.O...O.O#.#
###.#.#####.#.#
#O.O#O...O#.#.#
#.#.#.###.#.#.#
#O.O.O#O.O#.#.#
#.###.#.#.#.#.#
#O..#O.O.O#O.O#
###############
###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#x..#.....#...#
###############

###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#x....#...#.#.#
#.###.#.#.#.#.#
#x..#.....#...#
###############

###############
#.......#....E#
#.#.#

## References

- [A* Pathfinding Visualization Tutorial](https://www.youtube.com/watch?v=JtiK0DOeI4A)